In [1]:
import utils
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

wdir = 'X:/user/liprandicn/mt-comparison'

In [2]:
# Load data
zhao_df = pd.read_excel(f'{wdir}/Zhao2021/output/zhao2021_tab1.xlsx', header=[0,1], index_col=0)
regions_class = pd.read_csv(f'{wdir}/data/region_classification.csv')
regions_class = regions_class.groupby(['ISO3', 'gbd_level3', 'UN_M49_level1', 'IMAGE26', 'continents', 'gbd_location_id']).first().reset_index()
gbd_df = pd.read_csv(f'{wdir}/burkart2022/data/gbd_mortality/IHME-GBD_2021_DATA.csv')

### All ages comparisson

In [7]:
app1 = utils.load_burkart(wdir, regions_class, '1erf_', gbd_df, 'all', ['All ages'], continents=True)
app2 = utils.load_burkart(wdir, regions_class, '', gbd_df, 'all', ['All ages'], continents=True)
app3 = utils.load_burkart(wdir, regions_class, 'extrap_', gbd_df, 'all', ['All ages'], continents=True)
app4 = utils.load_burkart(wdir, regions_class, '1erf_extrap_', gbd_df, 'all', ['All ages'], continents=True)
carleton_all = utils.load_carleton(wdir, regions_class, t_type='All')

In [ ]:
df0 = zhao_df.xs('overall', axis=1, level=0).iloc[1:,:]
df1 = app1.set_index('continents').loc[df0.index]
df2 = app2.set_index('continents').loc[df0.index]
df3 = app3.set_index('continents').loc[df0.index]
# df4 = app4.set_index('continents').loc[df0.index]
df5 = carleton_all.set_index('continents').loc[df0.index]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df0.index,
    y=df0["mean"],
    mode="markers",
    name = 'Zhao 2021',
    error_y=dict(
        type="data",
        symmetric=False,
        array=df0["p95"] - df0["mean"],
        arrayminus=df0["mean"] - df0["p5"]
    ),
    marker=dict(size=10, color="cadetblue")
))

dfs = [df1, df2, df5]
names = ['GBD (1ERF)', 'GBD', 'Carleton 2022']
colors = ['lightcoral', 'sienna', 'grey']

for df, name, color in zip(dfs, names, colors):

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mean"],
        mode="markers",
        name = name,
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper_est"] - df["mean"],
            arrayminus=df["mean"] - df["lower_est"]
        ),
        marker=dict(size=10, color=color)
    ))


fig.update_layout(
    title="Estimation including 5–95% range",
    yaxis_title="Average annual deaths (2000-2019)",
    xaxis_title="Continent",
    height=500,
    width=700,
)

fig.show()

### Estimation for +65

In [ ]:
app1 = utils.load_burkart(wdir, regions_class, '1erf_', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
app2 = utils.load_burkart(wdir, regions_class, '', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
app3 = utils.load_burkart(wdir, regions_class, 'extrap_', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
app4 = utils.load_burkart(wdir, regions_class, '1erf_extrap_', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
carleton_oldest = utils.load_carleton(wdir, regions_class, t_type='All', age_group='oldest')
honda = utils.load_honda(wdir, regions_class, gbd_df, '2010', 'all', '2000-2019', 'continents')

In [ ]:
df0 = honda.set_index('continents')
df1 = app1.set_index('continents').loc[df0.index]
df2 = app2.set_index('continents').loc[df0.index]
df3 = app3.set_index('continents').loc[df0.index]
df5 = carleton_oldest.set_index('continents').loc[df0.index]

fig = go.Figure()

dfs = [df0, df1, df2, df5]#, df4]
names = ['Honda 2014', 'GBD (1ERF)', 'GBD', 'Carleton 2022']
colors = ['darkslateblue', 'lightcoral', 'sienna', 'grey']#, 'darkorange']  'rosybrown',

for df, name, color in zip(dfs, names, colors):

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mean"],
        mode="markers",
        name = name,
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper_est"] - df["mean"],
            arrayminus=df["mean"] - df["lower_est"]
        ),
        marker=dict(size=10, color=color)
    ))


fig.update_layout(
    title="Estimation including 5–95% range",
    yaxis_title="Average annual deaths (2000-2019)",
    xaxis_title="Continent",
    height=500,
    width=700,
)

fig.show()

### Spatial differences

In [8]:
regions = pd.read_csv(f'{wdir}/data/region_classification.csv')
locations_shp = gpd.read_file('X:\\user\\liprandicn\\mt-comparison\\data\\gbd_locations_shp\\GBD_shapefile.shp')

In [ ]:
temp_type = 'hot'
app1 = utils.load_burkart(wdir, regions_class, '1erf_', gbd_df, temp_type, ['All ages'], continents=False)
app2 = utils.load_burkart(wdir, regions_class, '', gbd_df, temp_type, ['All ages'], continents=False)
app3 = utils.load_burkart(wdir, regions_class, 'extrap_', gbd_df, temp_type, ['All ages'], continents=False)
app4 = utils.load_burkart(wdir, regions_class, '1erf_extrap_', gbd_df, temp_type, ['All ages'], continents=False)

app1 = locations_shp.merge(app1[['year', 'location_id', 'location_name', 'mean']], left_on='loc_id', right_on='location_id')
app1 = app1.drop_duplicates()
app2 = locations_shp.merge(app2[['year', 'location_id', 'location_name', 'mean']], left_on='loc_id', right_on='location_id')
app2 = app2.drop_duplicates()
app3 = locations_shp.merge(app3[['year', 'location_id', 'location_name', 'mean']], left_on='loc_id', right_on='location_id')
app3 = app3.drop_duplicates()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

cmap = 'inferno'
norm = mpl.colors.Normalize(vmin=0, vmax=1e3)

app1.plot(column='mean', cmap=cmap, norm=norm, ax=axes[0])
axes[0].set_title("1 function")
axes[0].axis('off')

app2.plot(column='mean', cmap=cmap, norm=norm, ax=axes[1])
axes[1].set_title("Original")
axes[1].axis('off')

# app3.plot(column='mean', cmap=cmap, norm=norm, ax=axes[2])
# axes[2].set_title("Extrapolation")
# axes[2].axis('off')

plt.subplots_adjust(wspace=-0.05, bottom=0.15) 

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm._A = []

cbar_ax = fig.add_axes([0.35, 0.28, 0.3, 0.025])  
cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar.set_label("Annual average deaths (2000-2019)")

plt.show()

### Comparison Honda Lancet 2024
Comparison with section 1.1.5.2 from the Appendix. Available at: 

In [ ]:
honda_df = pd.read_csv(f'{wdir}\\honda2014\\output\\paf_era5_GBD_level3_2000-2023_ot-2005_lancet_mean-t.csv', index_col=0, header=[0,1])
honda_df = honda_df.xs('hot', level=1, axis=1)
honda_df = honda_df.iloc[:,1:11].mean(axis=1).reset_index()

gbd_ = gbd_df[
        (gbd_df['cause_name']=='All causes') &
        (gbd_df['sex_name'] == 'Both') &
        (gbd_df['year'].isin(range(2001, 2011))) &
        (gbd_df['age_name'].isin(['65-69 years', '70-74 years', '75-79 years', '80-84 years', '85+ years'])) &
        (gbd_df['location_name'] != 'Global')][['location_id', 'location_name', 'cause_name', 'year', 'val', 'upper', 'lower']]	
gbd_['year']=gbd_['year'].astype(int)
gbd_= gbd_.groupby(['location_id', 'location_name', 'cause_name', 'year']).sum().reset_index().groupby(['location_id', 'location_name', 'cause_name']).mean().reset_index()  
gbd_= gbd_.merge(honda_df, left_on='location_id', right_on='index')
gbd_ = gbd_.merge(regions_class[['gbd_location_id', 'who_names']], left_on='location_id', right_on='gbd_location_id', how='left')
gbd_['mortality'] = gbd_['val']*gbd_[0]*0.88
gbd_.groupby('who_names')['mortality'].sum().reset_index()#.sum(axis=0)

In [ ]:
year = 2021

honda_df = pd.read_csv(f'{wdir}\\honda2014\\output\\paf_era5_GBD_level3_2021-2023_ot-2005_lancet.csv', index_col=0, header=[0,1])
honda_df = honda_df.xs('hot', level=1, axis=1)
honda_df = honda_df.loc[:,f'{year}'].reset_index()

gbd_ = gbd_df[
        (gbd_df['cause_name']=='All causes') &
        (gbd_df['sex_name'] == 'Both') &
        (gbd_df['year'].isin(range(year,year+1))) &
        (gbd_df['age_name'].isin(['65-69 years', '70-74 years', '75-79 years', '80-84 years', '85+ years'])) &
        (gbd_df['location_name'] != 'Global')][['location_id', 'location_name', 'cause_name', 'year', 'val', 'upper', 'lower']]	
gbd_['year']=gbd_['year'].astype(int)
gbd_ = gbd_.groupby(['location_id', 'location_name', 'cause_name', 'year']).sum().reset_index().groupby(['location_id', 'location_name', 'cause_name']).mean().reset_index()  
gbd_ = gbd_.merge(honda_df, left_on='location_id', right_on='index')
gbd_ = gbd_.merge(regions_class[['gbd_location_id', 'who_names']], left_on='location_id', right_on='gbd_location_id')
gbd_['mortality'] = gbd_['val']*gbd_[f'{year}']*0.88
gbd_.groupby('who_names')['mortality'].sum().reset_index()

In [ ]:
gbd_['mortality'].sum()

In [ ]:
honda_df = pd.read_csv(f'{wdir}\\honda2014\\output\\paf_era5_GBD_level3_2000-2023_ot-2005_lancet.csv', index_col=0, header=[0,1])
honda_df = honda_df.xs('hot', level=1, axis=1)
honda_df = honda_df.loc[:,'2013'].reset_index()

gbd_ = gbd_df[
        (gbd_df['cause_name']=='All causes') &
        (gbd_df['sex_name'] == 'Both') &
        (gbd_df['year'].isin(range(2013,2014))) &
        (gbd_df['age_name'].isin(['65-69 years', '70-74 years', '75-79 years', '80-84 years', '85+ years'])) &
        (gbd_df['location_name'] != 'Global')][['location_id', 'location_name', 'cause_name', 'year', 'val', 'upper', 'lower']]	
gbd_['year']=gbd_['year'].astype(int)
gbd_ = gbd_.groupby(['location_id', 'location_name', 'cause_name', 'year']).sum().reset_index().groupby(['location_id', 'location_name', 'cause_name']).mean().reset_index()  
gbd_ = gbd_.merge(honda_df, left_on='location_id', right_on='index')
gbd_ = gbd_.merge(regions_class[['gbd_location_id', 'continents']], left_on='location_id', right_on='gbd_location_id')
gbd_['mortality'] = gbd_['val']*gbd_['2013']
gbd_[gbd_['continents']=='Europe'][['location_name', 'mortality']].sort_values(by='location_name', ascending=True).reset_index(drop=True)

### Montlhy statistics

In [ ]:
region_name = 'Belize'

honda_era5 = pd.read_csv(f'{wdir}/honda2014/output/honda_paf_countries_ERA5_2000-2024_ot-2010.csv', header=[0,1], index_col=0)
honda_ms = pd.read_csv(f'{wdir}/honda2014/output/honda_paf_countries_MS_2000-2024_ot-2010.csv', header=[0,1], index_col=0)

region = regions_class[regions_class['gbd_level3']==region_name]['gbd_location_id'].values[0]

for type, color in zip(['cold', 'hot', 'all'], ['C0', 'C1', 'C2']):
    plt.plot(honda_era5.xs(type, level=1, axis=1).loc[region,:], label=f'ERA5 {type}', color=color)
    plt.plot(honda_ms.xs(type, level=1, axis=1).loc[region,:], label=f'MS {type}', linestyle='--', color=color) 
    
plt.xticks(range(0,25,3), range(2000,2025,3))
plt.ylabel('Population Attributable Fraction (%)')
plt.title(f'Honda | {regions_class[regions_class['gbd_location_id']==region]['gbd_level3'].values[0]}')
plt.legend()
plt.show() 

In [ ]:
region_name = 'Netherlands'

burkart_era5 = pd.read_csv(f'{wdir}/burkart2022/output/burkart_paf_countries_ERA5_2000-2019.csv', header=[0,1,2], index_col=0)
burkart_ms = pd.read_csv(f'{wdir}/burkart2022/output/burkart_paf_countries_MS_2000-2019.csv', header=[0,1,2], index_col=0)

region = regions_class[regions_class['gbd_level3']==region_name]['gbd_location_id'].values[0]

for type, color in zip(['cold', 'hot', 'all'], ['C0', 'C1', 'C2']):
    plt.plot(burkart_era5.xs('ckd', level=1, axis=1).xs(type, level=1, axis=1).loc[region,:], label=f'ERA5 {type}', color=color)
    plt.plot(burkart_ms.xs('ckd', level=1, axis=1).xs(type, level=1, axis=1).loc[region,:], label=f'MS {type}', linestyle='--', color=color) 
    
plt.xticks(range(0,25,3), range(2000,2025,3))
plt.ylabel('Population Attributable Fraction (%)')
plt.title(f'Burkart | {regions_class[regions_class['gbd_location_id']==region]['gbd_level3'].values[0]}')
plt.legend()
plt.show() 

In [ ]:
import rioxarray 
countries = gpd.read_file(wdir+'/burkart2022/data/regions_classification/GBD/gbd_shapefiles/GBD_shapefile.shp')
country = countries[countries['loc_name']==region_name]

temp = xr.open_dataset('X:/user/scherrenbm/ModelDevelopment/IMAGE_Development/IMAGE_Daily_Indicators/SSP2/netcdf/GTMP_MEAN_30MIN.nc')
da = temp.GTMP_MEAN_30MIN.isel(time=-1).isel(NM=0)
da = da.rio.write_crs("EPSG:4326")

da_country = da.rio.clip(country.geometry, country.crs)

fig, ax = plt.subplots()
da_country.plot(ax=ax, add_colorbar=False)
country.boundary.plot(ax=ax, color='black', linewidth=2)
plt.show()